In [2]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
import deepxde as dde
import numpy as np

In [3]:
# Problem parameters:
a = 0.4  # Thermal diffusivity
L = 1  # Length of the bar
n = 1  # Frequency of the sinusoidal initial conditions

def pde(x, y):
    """Expresses the PDE residual of the heat equation."""
    dy_t = dde.grad.jacobian(y, x, i=0, j=1)
    dy_xx = dde.grad.hessian(y, x, i=0, j=0)
    return dy_t - a * dy_xx

# Computational geometry:
geom = dde.geometry.Interval(0, L)
timedomain = dde.geometry.TimeDomain(0, 1)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Initial and boundary conditions:
bc = dde.icbc.NeumannBC(geomtime, lambda x: 0, lambda _, on_boundary: on_boundary)
ic = dde.icbc.IC(
    geomtime,
    lambda x: np.cos(np.pi * x[:, 0:1] / L),
    lambda _, on_initial: on_initial,
)

# Define the PDE problem and configurations of the network:
data = dde.data.TimePDE(
    geomtime,
    pde, # minimize residual
    [bc, ic],
    num_domain=2540,
    num_boundary=80,
    num_initial=160,
    num_test=2540,
)
net = dde.nn.FNN([2] + [10] * 2 + [1], "tanh", "Glorot normal")
model = dde.Model(data, net)

In [4]:
# Build and train the model:
model.compile("adam", lr=1e-3)
model.train(iterations=10000)

Compiling model...
Building feed-forward neural network...
'build' took 0.121782 s



C:\Users\reloj\anaconda3\lib\site-packages\deepxde\nn\tensorflow_compat_v1\fnn.py:114: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


'compile' took 1.113064 s

Initializing variables...
Training model...

Step      Train loss                        Test loss                         Test metric
0         [2.08e-02, 2.78e-02, 4.45e-01]    [2.07e-02, 2.78e-02, 4.45e-01]    []  
1000      [2.21e-03, 6.34e-04, 1.90e-04]    [1.36e-03, 6.34e-04, 1.90e-04]    []  
2000      [4.42e-04, 4.48e-05, 1.00e-05]    [3.23e-04, 4.48e-05, 1.00e-05]    []  
3000      [2.78e-04, 2.18e-05, 3.28e-06]    [2.05e-04, 2.18e-05, 3.28e-06]    []  
4000      [1.98e-04, 1.15e-05, 2.10e-06]    [1.42e-04, 1.15e-05, 2.10e-06]    []  
5000      [1.53e-04, 7.16e-06, 1.61e-06]    [1.07e-04, 7.16e-06, 1.61e-06]    []  
6000      [1.27e-04, 5.60e-06, 1.65e-06]    [8.88e-05, 5.60e-06, 1.65e-06]    []  
7000      [9.75e-05, 3.48e-06, 1.07e-06]    [6.37e-05, 3.48e-06, 1.07e-06]    []  
8000      [8.12e-05, 3.11e-06, 8.53e-07]    [5.23e-05, 3.11e-06, 8.53e-07]    []  
9000      [7.02e-05, 2.94e-06, 7.49e-07]    [4.45e-05, 2.94e-06, 7.49e-07]    []  
10000   

(<deepxde.model.LossHistory at 0x234c5e8c5b0>,
 <deepxde.model.TrainState at 0x234b78dcc40>)

In [5]:
model.print_model()

Variable: dense/kernel:0, Shape: (2, 10)
[[-1.5990883   0.37469533  0.45382783  1.4474     -0.07592233 -0.52672696
   0.4404916  -0.5205638   0.20294623 -1.4355215 ]
 [ 0.35831878 -0.7573527   1.004971    1.1180861   1.0793487  -0.39940417
  -1.1148055   0.8970601   1.2398698  -0.6011251 ]]
Variable: dense/bias:0, Shape: (10,)
[ 0.48147652 -0.36868328  0.03046252 -0.5381217   0.44963765 -0.30798987
 -0.17150135  0.33380023  0.27705324  0.0529801 ]
Variable: dense_1/kernel:0, Shape: (10, 10)
[[ 1.0536628  -0.49546582  0.07132808 -0.74095696  0.03663914 -0.2922958
  -0.4089138   0.8712825  -0.06003215 -0.2738065 ]
 [-0.91908675 -2.3150625  -0.8240347   0.33802855  0.46728927  0.5489989
  -0.6502275  -0.21120869  0.22945999  0.20071827]
 [ 1.2934774  -0.6968734   0.75158674 -0.9747271   1.0177298   0.1179947
  -1.0861077  -0.7476071  -0.5889487  -0.27543148]
 [-0.21390183  0.7311661   0.20615576  0.75262904 -0.26487216 -0.19175258
   0.75638336  0.14215621 -0.3580125   0.09616257]
 [ 1.20